# Contents

* [Read Filtering](#Read-Filtering)
* [Result Evaluation](#Result-Evaluation)

# Read Filtering

The code in this section describes how the simulated set of reads is filtered to exclude all reads for which edlib finds more than 20 mapping positions.

###### Requirements:
* Set of simulated reads (`simulations/reads/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10.fasta`)
* edlib mapping results for whole read set (`simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri0-69400.er`)

In [ ]:
from Bio import SeqIO

edlibResultPath = "simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0" + \
"009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri0-69400.er"
rarelyMappedReadIds = {}
readId = -1
resultCounter = 0

for l in open(edlibResultPath, 'r'):
    if l.startswith("simulations/edlibMappings/"):
        if readId > -1 and resultCounter <= 20:
            rarelyMappedReadIds[readId] = None
            
        readId = int(l.split("_ri")[1].split(".er")[0])
        resultCounter = 0
    else:
        resultCounter += 1
        
if resultCounter <= 20:
    rarelyMappedReadIds[readId] = None
    
allReadsPath = "simulations/reads/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.000909090909" + \
"0909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10.fasta"
    
rarelyMappedReadRecords = [r for r in SeqIO.parse(allReadsPath, "fasta") if int(r.id) in rarelyMappedReadIds]
rarelyMappedReadsFilePath = "simulations/reads/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0" + \
".0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20.fasta"
SeqIO.write(rarelyMappedReadRecords, open(rarelyMappedReadsFilePath, 'w'), "fasta")

# Result Evaluation

The code in this section allows to perform a mapping accuracy evaluation using BLAST confirmed mapping results as the ground truth.

###### Requirements:
* T2T assembly of human chromosome Y (`simulations/genomes/t2thumanChrY.fasta`)
* Filtered set of simulated reads (`simulations/reads/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20.fasta`)
* List of k-mers occurring more than 100 times (`highAbundKmersMiniK15w10Lrgr100BtStrnds.txt`)
* edlib mapping results for whole read set (`simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri0-69400.er`)
* eskemap mapping results for filtered read set (`simulations/homologies/homologies_t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20_k15_w10_c1_u1_de0.08758516_in-231.1585158515809.txt`)
* minimap2 mapping results (`simulations/minimap2Res/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20_k15.paf.gz`)
* Winnowmap2 mapping results (`simulations/Winnowmap2Res/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20_k15.paf.gz`)
* BLAST results for mapping results (`simulations/blastRes/*_e0.01.tsv`)

In [ ]:
from Bio import SeqIO
from collections import deque
import gzip

NT_IN_BITS = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

#Read FASTA file
def readFasta(path):
    return [r for r in SeqIO.parse(open(path, 'r'), "fasta")]

#Get a subset of all edlib results that is based on a maximum number of results per read
def createEdlibResSubsetBasedOnMaxResNbPerRead(allEdlibRes, maxResNb):
    subSet = {}
    
    for r in allEdlibRes:
        if len(allEdlibRes[r]) <= maxResNb:
            subSet[r] = list(allEdlibRes[r])
            
    return subSet

#This function filters edlib's results based on a distance threshold
#Parameters:
#    res: A dictionary of edlib results with integer-based read ids as keys and a list representing ...
#    thres: Relative distance threshold
def filterByDistanceThreshold(res, thres):
    filteredRes = {}
    
    for r in res:
        filteredRes[r] = []
        absThres = thres * len(rdSeqs[f"s_{r}"].seq)
        
        for rs in res[r]:
            if int(rs[-1]) <= absThres:
                filteredRes[r].append(rs)
        
    return filteredRes

#This function calculates the hash of a bitwise k-mer representation. The function is influenced by the code of "The
#minimizer Jaccard estimator is biased and inconsistent." from Belbasi et al. (function "minimap2_hash(seed,v,mask)"
#in file "minimap2_hash_uncompiled.py").
def getHash(kmer, mask):
    u = kmer & mask
    u = ((~u) + (u << 21)) & mask # u = (u<<21)-(u+1) = 77594587*u-1
    u = u ^ (u >> 24)
    u = ((u + (u << 3)) + (u << 8)) & mask # u *= 265
    u = u ^ (u >> 14)
    u = ((u + (u << 2)) + (u << 4)) & mask # u *= 21
    u = u ^ (u >> 28)
    u = (u + (u << 31)) & mask # u *= 2147483649

    return u

#This function calculates the minimizer sketch of a sequence. It is influenced by the code of "The minimizer Jaccard
#estimator is biased and inconsistent." from Belbasi et al. (function "winnowed_minimizers_linear(perm,windowSize)" 
#in file "winnowed_minimizers.py").
def calcMiniSketch(seq, k, w):
    sketch = []
    #A deque to store k-mer hashes inside the current window
    windowKmers = deque()
    mask = (4 ** k) - 1
    lastIdx = -1

    for i in range(len(seq) - k + 1):
        kmerBits = 0
        kmerBitsRevComp = 0
        windowBorder = i - (w - 1)
        
        #Get bit representation of k-mer
        for c in seq[i:i+k]:
            kmerBits = (kmerBits << 2) + NT_IN_BITS[c]

        #Get bit representation of k-mer's reverse complement
        for c in str(Seq(seq[i:i+k]).reverse_complement()):
            kmerBitsRevComp = (kmerBitsRevComp << 2) + NT_IN_BITS[c]

        #If a k-mer is its own reverse complement we skip it
        if kmerBits == kmerBitsRevComp:
            continue

        #Depending on which hash is smaller we consider either a k-mer or its reverse complement per position
        if kmerBits < kmerBitsRevComp:
            kmer = (i, kmerBits, getHash(kmerBits, mask))
        else:
            #A k-mer is a pair of k-mer's start position and its hash
            kmer = (i, kmerBitsRevComp, getHash(kmerBitsRevComp, mask))
            
        #Remove all k-mers with a hash value larger than the newly calculated one
        while (len(windowKmers) > 0) and (windowKmers[-1][2] > kmer[2]):
            windowKmers.pop()

        #Save new k-mer as window k-mer
        windowKmers.append(kmer)

        #Remove k-mer if it is not any longer inside the window
        while (len(windowKmers) > 0) and (windowKmers[0][0] < windowBorder):
            windowKmers.popleft()

        #As soon as we have seen a first full window of k-mers choose a minimizer
        if (windowBorder >= 0) and (len(windowKmers) > 0):      
            #We do not choose the same minimizer for a second time
            if lastIdx != windowKmers[0][0]:
                sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))
                lastIdx = windowKmers[0][0]
                
            while len(windowKmers) > 1 and windowKmers[0][1] == windowKmers[1][1]:
                windowKmers.popleft()
                sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))    
                lastIdx = windowKmers[0][0] 

    #If our sequence was too small to get a full window of k-mers to consider take the smallest one found so far
    if windowBorder < 0 and len(windowKmers) > 0:
        sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))
        
        while len(windowKmers) > 1 and windowKmers[0][1] == windowKmers[1][1]:
            windowKmers.popleft()
            sketch.append((windowKmers[0][0]+k-1, windowKmers[0][1], windowKmers[0][2]))

    return sketch

def loadFindThomsRes(pth, refSketch):
    compResPerRead = {}
    
    for l in open(pth, 'r'):
        l = l.strip()

        if l.startswith('s'):
            lastRdId = int(l.split('_')[1])
            compResPerRead[lastRdId] = []
        else:
            cols = l.split(' ')
            s = refSketch[int(cols[1])][0] + 1 - K
            e = refSketch[int(cols[3])][0]
            compResPerRead[lastRdId].append({"id": lastRdId, "start": s, "end": e, "score": float(cols[5])})
        
    return compResPerRead

#This function returns a dictionary with integer-based read ids as keys and a list of start stop tupels of homology 
#intervals reported by FindThoms as values. Stop coordinates may be corrected in the way that the k-1 overlap of the
#last k-mer part of the alpha-homology is not included anymore.
#Parameter: 
#res: Result of FindThoms as returned from function loadFindThomsRes
#corrEnd: Flag indicating if the end coordinate shall be adaptated
def parseCoordsFromFindThomsRes(res, corrEnd):
    ints = {}
        
    for r in res:
        if corrEnd:
            #Start position was corrected inside loadFindThomsRes already
            ints[r] = [(h["start"], h["end"] + 1 - 15) for h in res[r]]
        else:
            ints[r] = [(h["start"], h["end"]) for h in res[r]]
        
    return ints

#This function filters FindThom's results for intervals above a threshold score relative to read lengths and 
#interpolated using given decent and intercept
#Parameters:
#    res: A dictionary of results as returned by loadFindThomsRes(pth, refSketch)
#    dec: Float describing the decent of the line used for threshold interpolation
#    inter: Float describing the intercept of the line used for threshold interpolation
def filterByScoreThreshold(res, dec, inter):
    filteredRes = {}
    
    for r in res:
        filteredRes[r] = []
        absThres = dec * len(rdSeqs[f"s_{r}"].seq) + inter
        
        for rs in res[r]:
            if rs["score"] >= absThres:
                filteredRes[r].append(rs)
                
    return filteredRes

def parseCompressedPAF(filepath):
    resPerRead = {}
    
    for l in gzip.open(filepath, 'rt'):
        elems = l.strip().split('\t')
        rdid = int(elems[0].split('_')[1])
        
        if rdid in resPerRead:
            resPerRead[rdid].append((int(elems[7]), int(elems[8]) - 1))
        else:
            resPerRead[rdid] = [(int(elems[7]), int(elems[8]) - 1)]
            
    return resPerRead

#This function reads BLAST results given a list of BLAST result files and adds it to a dictionary
#Parameters:
#    resFilePaths: List of result file paths
#    blastRes: A dictionary assumed to be shaped as described above
def loadBLASTresults(resFilePaths, blastRes):
    for f in resFilePaths:
        for l in open(f, 'r'):
            #Header line
            if not l.startswith("s_"):
                #Find out read id
                idPattern = search("s_[0-9]+", l)
                #Find out reference substring range
                rangePattern = search("ref[0-9]+-[0-9]+", l)
                #Parse start and end of reference substring
                subStart, subEnd = [int(c) for c in rangePattern.group(0).split("ref")[1].split('-')]
                #Parse read id
                readId = int(idPattern.group(0).split("s_")[1])

                #Check if this read has been seen before and add it if necessary
                if not readId in blastRes:
                    blastRes[readId] = {}
                    
                #Check if even this substring has been seen before for this read
                if not (subStart, subEnd) in blastRes[readId]:
                    blastRes[readId][(subStart, subEnd)] = []
            else:
                elems = l.split('\t')
                
                #Sanity check
                #This won't work for FindThoms results since I messed up FASTA headers there...
                if len(elems[3].split('-')) < 2:
                    hitRefStart = subStart
                    hitRefEnd = subEnd
                else:
                    #Check if reference substring in the header matches that of the current hit
                    hitRefStart, hitRefEnd = [int(c) for c in elems[3].split("ref")[1].split('-')]
                
                if hitRefStart != subStart or hitRefEnd != subEnd:
                    print(f"Some result for read s_{readId} in file {f} has different substring coordinates as " + \
                          "expected from the header")
                    print("Hit coordinates:", hitRefStart, hitRefEnd)
                    print("Coordinates in header:", subStart, subEnd)
                    
#                     p0 = f"../simulations/mappedAreas1/otherTools/sub_s_{readId}_ref{subStart}-{subEnd}.fasta"
#                     p1 = f"../simulations/mappedAreas1/otherTools/sub_s_{readId}_ref{hitRefStart}-{hitRefEnd}" + \
#     ".fasta"
        
#                     if not exists(p0):
#                         SeqIO.write(SeqRecord(Seq(refSeq[subStart:subEnd+1]), id=f"ref{subStart}-{subEnd}"), \
#                          0           open(p0, 'w'), "fasta")
                    
#                     if not exists(p1):
#                         SeqIO.write(SeqRecord(Seq(refSeq[hitRefStart:hitRefEnd+1]), id=f"ref{hitRefStart}-" + \
#                                               f"{hitRefEnd}"), open(p1, 'w'), "fasta")

                    continue
                
                #Filter for evalue just to be sure
                if float(elems[6]) <= 0.01:
                    blastRes[readId][(subStart, subEnd)].append({"rdStart": int(elems[1]), "rdEnd": int(elems[2]), \
                                                                 "refStart": int(elems[4]), "refEnd": int(elems[5])\
                                                                 , "strand": elems[13], "qcov": int(elems[14]), \
                                                                 "eval": float(elems[6])})
                    
#Generate BLAST result subsets according to e-value thresholds
def filterByEval(res, ethr):
    filteredRes = {}
    
    for r in res:
        filteredRes[r] = {}
        
        for s in res[r]:
            for f in res[r][s]:
                if f["eval"] <= ethr:
                    if s in filteredRes[r]:
                        filteredRes[r][s].append(f)
                    else:
                        filteredRes[r][s] = [f]
                        
    return filteredRes

#Filter BLAST results by e-value
def filterByEvalRng(res, evals):
    fltRes = {}
    
    for t in evals:
        fltRes[t] = filterByEval(res, t)
        
    return fltRes

#Form BLAST result subsets for tool result subsets we want to evaluate
def filterByRes(totRes, filtRes):
    blastSubsetRes = {}
    
    #For each tool
    for t in filtRes:
        for r in filtRes[t]:
            for f in filtRes[t][r]:
                if f in totRes[r]:
                    if r in blastSubsetRes:
                        if not f in blastSubsetRes[r]:
                            blastSubsetRes[r][f] = totRes[r][f]
                    else:
                        blastSubsetRes[r] = {f: totRes[r][f]}
            
    return blastSubsetRes

#Generate BLAST result subsets based on tool thresholds
def genBlResSubsFrTlRes(tlThrBlRsDict, res2Flt, tlResCrds2Flt, tlNm):
    thresholds = sorted(tlThrBlRsDict.keys(), reverse=True)
    
    for t in thresholds[1:]:
        res2Flt[tlNm] = tlResCrds2Flt[t]
        tlThrBlRsDict[t] = filterByRes(tlThrBlRsDict[thresholds[0]], res2Flt)
        
#Generate ground truth data set for result subsets
def detCorrMappings(bRes):
    corrMappings = {}

    for r in bRes:
        corrMappings[r] = []
        rdLen = len(rdSeqs[f"s_{r}"].seq)

        for i in bRes[r]:
            subStrLen = i[1] - i[0] + 1
            partRes = {"minus": [], "plus": []}
            noRes = True

            for br in bRes[r][i]:
                covSubStr = abs(br["refEnd"] - br["refStart"] + 1)
            
                if covSubStr >= 0.9 * subStrLen or br["qcov"] >= 90:
                    corrMappings[r].append(i)
                    noRes = False
                    break
                else:
                    #TODO: This can be done better
                    if br["strand"] == "plus":
                        start = br["refStart"]
                        end = br["refEnd"]
                    else:
                        start = br["refEnd"]
                        end = br["refStart"]

                    ovl = False

                    for intv in partRes[br["strand"]]:
                        if (start <= intv[0] and end >= intv[0]) or (start <= i[1] and end >= i[1]):
                            ovl = True
                            break

                    if not ovl:
                        partRes[br["strand"]].append((start, end))
                    
            if noRes:
                lenResParts = [0, 0]

                for j, s in enumerate(partRes):
                    for ints in partRes[s]:
                        lenResParts[j] += ints[1] - ints[0] + 1

                if max(lenResParts) >= 0.9 * subStrLen or max(lenResParts) >= 0.9 * rdLen:
                    corrMappings[r].append(i)
        
    return corrMappings

#This function returns a dictionary of k-mers from the given reference sketch that are covered by the given results 
#of some tool per each read from the given set of read ids.
#refSk: A list of start positions of k-mers in a sequence that are chosen as being part of the sketch
#res: A dictionary with read ids as keys and lists of start and stop coordinate tuples as values
def getCovKmers(refSk, res):
    covKmers = {}
    #Sort start positions in sketch
    sortedRefSk = sorted(refSk)
    
    #Iterate over reads:
    for r in res:
        #Initialize dictionary for this read
        covKmers[r] = {}
        
        #If there are no results we are done for this read
        if len(res[r]) == 0:
            continue
            
        #Initialize a flag marking that we have reached the end of our result list
        endReached = False
        #Sort result coordinates for this read increasingly by start position
        coords = sorted(res[r], key=lambda e: e[0])
        #Initialize an index to know which result positions we consider currently (we start with the one that has 
        #the smallest start coordinate)
        i = 0
        
        #Iterate over sorted list of start positions
        for p in sortedRefSk:
            #We need to switch to a result which has a larger start position as long as the position of the refer-
            #ence sketch k-mer is larger than the end position of our current result
            while p > coords[i][1]:
                #Move to the next result
                i += 1
                
                #Check if there is another result at all
                if i >= len(coords):
                    endReached = True
                    break
                    
            #Check if we set the flag
            if endReached:
                break
                
            #Check if the k-mer starts before the current result
            if p < coords[i][0]:
                continue
                
            #If we have reached this point the k-mer must fall inside the result
            covKmers[r][p] = None
        
    return covKmers

#This function creates a dictionary with integer-based read ids as keys and lists of alignment coordinates as values
#from a given dictionary of edlib alignment results
#Parameter: res: Dictionary of edlib alignment results with integer-based read ids as keys and lists of alignment
#                infos parsed from file
def getEdlibResAlnCrds(res):
    return {r: [(int(rs[0]), int(rs[1])) for rs in res[r]] for r in res}

#This function calculates the accurracy of a tool for a given reference sketch and result files of edlib and 
#the tool using the "ref sketch method"
def calcFindThomsAccRefSkMeth(refSk, edlibRes, compRes):
    tpFpTnFnDict = {"TPs": 0, "FPs": 0, "TNs": 0, "FNs": 0}
    #Get number of all k-mers in reference sketch
    nbAllKmers = len(refSk)
    
    #Testing
    tps = 0
    
    #Iterate over read ids
    for r in edlibRes:
        #Iterate over k-mer positions
        for p in edlibRes[r]:
            #We have a TP if the k-mer is covered by both results
            if r in compRes and p in compRes[r]:
                tpFpTnFnDict["TPs"] += 1
            else:
                tpFpTnFnDict["FNs"] += 1
                
        if r in compRes:
            #Iterate over k-mer positions
            for p in compRes[r]:
                #Count TPs again as a sanity check
                if p in edlibRes[r]:
                    tps += 1
                else:
                    tpFpTnFnDict["FPs"] += 1
                
    #Testing
    if not tps == tpFpTnFnDict["TPs"]:
        print("Sanity check failed\nTP counts are different")
        
    tpFpTnFnDict["TNs"] = (len(edlibRes) * nbAllKmers) - sum([tpFpTnFnDict[c] for c in tpFpTnFnDict])
        
    print(tpFpTnFnDict)
    
    return tpFpTnFnDict

In [ ]:
#Load reads
rarelyMappedReadsFilePath = "simulations/reads/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0" + \
".0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20.fasta"
rdSeqs = readFasta(rarelyMappedReadsFilePath)

In [ ]:
from glob import glob
from matplotlib import pyplot as plt
from matplotlib import colors
import numpy as np

#Load edlib results
edlibResultPath = "simulations/edlibMappings/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0" + \
"009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_ri0-69400.er"
edlibRes = {}

for l in open(edlibResultPath, 'r'):
    if l.startswith("simulations/edlibMappings/"):
        readId = int(l.split("_ri")[1].split(".er")[0])
        edlibRes[readId] = []
    else:
        edlibRes[readId].append(l.strip().split(' '))
        
#Create "rarely mapped 20" subset of edlib results
edlibResMaxRes = {}
edlibResMaxRes[20] = createEdlibResSubsetBasedOnMaxResNbPerRead(edlibRes, 20)

#Filter results based on distance thresholds
edlibThres = [0.01, 0.02, 0.03]
filteredEdlibRes = {0.03: edlibResMaxRes[20]}

for t in edlibThres[:2]:
    filteredEdlibRes[t] = filterByDistanceThreshold(edlibResMaxRes[20], t)
    
#Load blacklisted k-mers
K = 15
blKmers = {int(l): None for l in open("highAbundKmersMiniK15w10Lrgr100BtStrnds.txt", 'r')}
#Load reference sequence
refSeq = readFasta("simulations/genomes/t2thumanChrY.fasta")[0].seq
#Calculate minimizer sketch of reference without blacklisted k-mers
fltRfSkMiK15W10 = [k for k in calcMiniSketch(str(refSeq), K, 10) if k[2] not in blKmers]
fltRfSkMiKmrStPosK15W10 = [k[0] + 1 - K for k in fltRfSkMiK15W10 if k[2] not in blKmers]
#Load eskemap results
p = "simulations/homologies/homologies_t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i0.0009090" + \
"909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_rm20_k15_w10_c1_u1_de0.08758516_in-2" + \
"31.1585158515809.txt"
FindThomsNestResApprxMppngRr20 = loadFindThomsRes(p, fltRfSkMiK15W10)
#Generate FindThoms result subsets according to thresholds
FindThomsDecs = {0.9: 0.08790719, 0.8: 0.08817002, 0.7: 0.08841644}
FindThomsInters = {0.9: -193.79071907191974, 0.8: -110.81700170017575, 0.7: -60.04164416442654}
dpFltHomCrdsRr20 = parseCoordsFromFindThomsRes(FindThomsNestResApprxMppngRr20, False)
dpFltCrds = {0.95: dpFltHomCrdsRr20}

for i in FindThomsDecs:
    fltHoms = filterByScoreThreshold(FindThomsNestResApprxMppngRr20, FindThomsDecs[i], FindThomsInters[i])
    dpFltCrds[i] = parseCoordsFromFindThomsRes(fltHoms, False)
    
#Load competitors' results
pafRes = {}
relPafRes = {}

for t in ["minimap2", "Winnowmap2"]:
    p = f"simulations/{t}Res/t2thumanChrY_sr0.00010909090909090909_dr0.0009818181818181818_i" + \
    "0.0009090909090909091_sd7361077429744071834_lmn100_lmx1000000_lavg9000_ls7000_dp10_k15.paf.gz"
    pafRes[t] = parseCompressedPAF(p)
    #Only consider results of relevant reads to speed this up
    relPafRes[t] = {r: pafRes[t][r] for r in pafRes[t] if r in edlibResMaxRes[20]}
    
#Load BLAST results
blastResReload = {}
loadBLASTresults(glob("simulations/blastRes/*_e0.01.tsv"), blastResReload)
#Generate BLAST result subsets
evalthrs = [0.01, 0.005, 0.001]
filteredBlastRes = filterByEvalRng(blastResReload, evalthrs)

#Do I need this?
# edlibThrsAllHomBlstRs = {0.03: blastResReload, 0.02: None, 0.01: None}
# allHomRes2Flt =  {"ESKEMAP": dpFltHomCrdsRr20, "minimap2": relPafRes["minimap2"], "Winnowmap2": relPafRes\
#                   ["Winnowmap2"]}
# fltEdlibResCrds = {}

# for t in edlibThres:
#     fltEdlibResCrds[t] = getEdlibResAlnCrds(filteredEdlibRes[t])
    
# genBlResSubsFrTlRes(edlibThrsAllHomBlstRs, allHomRes2Flt, fltEdlibResCrds, "edlib")
# scrThrsAllHomBlstRs = {0.95: allHomBLASTres, 0.9: None, 0.8: None, 0.7: None}
# genBlResSubsFrTlRes(scrThrsAllHomBlstRs, allHomRes2Flt, dpFltCrds, "ESKEMAP")

#Generate ground truth sets
corrResVarEval = {}

for t in evalthrs:
    corrResVarEval[t] = detCorrMappings(filteredBlastRes[t])
    
# corrResAllHomVarDstThr = {}

#Do I need this?
# for t in edlibThres:
#     corrResAllHomVarDstThr[t] = detCorrMappings(edlibThrsAllHomBlstRs[t])
    
# corrResAllHomVarScrThr = {}
 
# for t in [0.95, 0.9, 0.8, 0.7]:
#     corrResAllHomVarScrThr[t] = detCorrMappings(scrThrsAllHomBlstRs[t])

#Calculate covered k-mers
covKmersCorrAllHomVarEval = {}

for t in evalthrs:
    covKmersCorrAllHomVarEval[t] = getCovKmers(fltRfSkMiKmrStPosK15W10, corrResVarEval[t])
    
covKmersEskemap = {}

for t in FindThomsDecs:
    covKmersEskemap[t] = getCovKmers(fltRfSkMiKmrStPosK15W10, dpFltCrds[i])
    
for t in edlibThres:
    covKmersEdlib[t] = getCovKmers(fltRfSkMiKmrStPosK15W10, getEdlibResAlnCrds(filteredEdlibRes[t]))

covKmersCompetitors = {}

for t in ["minimap2", "Winnowmap2"]:
    covKmersCompetitors[t] = getCovKmers(fltRfSkMiKmrStPosK15W10, relPafRes[t])

#Calculate mapping accuracies
tpfptnfnVarEval = {}

for t in evalthrs:
    tpfptnfnVarEval[t] = {}
    tpfptnfnVarEval[t]["edlib"] = calcFindThomsAccRefSkMeth(fltRfSkMiK15W10, corrResVarEval[t], covKmersEdlib[0.03])
    tpfptnfnVarEval[t]["eskemap"] = calcFindThomsAccRefSkMeth(fltRfSkMiK15W10, corrResVarEval[t], \
                                                              covKmersEskemap[0.95])

    for c in ["minimap2", "Winnowmap2"]:
        tpfptnfnVarEval[t][c] = calcFindThomsAccRefSkMeth(fltRfSkMiK15W10, corrResVarEval[t], \
                                                          covKmersCompetitors[c])
        
tpfptnfnVarScrThr = {}

for t in FindThomsDecs:
    tpfptnfnVarScrThr[t] = calcFindThomsAccRefSkMeth(fltRfSkMiK15W10, corrResVarEval[0.01], covKmersEskemap[t])
    
tpfptnfnVarDstThr = {}

for t in edlibThres:
    tpfptnfnVarDstThr[t] = calcFindThomsAccRefSkMeth(fltRfSkMiK15W10, corrResVarEval[0.01], covKmersEdlib[t])
        
#Make a plot for varying e-values
dotshapes = {"ESKEMAP": "o", "minimap2": "^", "Winnowmap2": "v", "edlib": "P"}
dotColors = ['b', 'r', 'g']
cnt = 0
plt.figure()

for et in evalthrs:
    for t in tpfptnfnVarEval[et]:    
        prec = tpfptnfnVarEval[et][t]["TPs"] / (tpfptnfnVarEval[et][t]["TPs"] + tpfptnfnVarEval[et][t]["FPs"])
        recall = tpfptnfnVarEval[et][t]["TPs"] / (tpfptnfnVarEval[et][t]["TPs"] + tpfptnfnVarEval[et][t]["FNs"])
        
        print("E-value:", et, "Tool:", t)
        print("Precision:", prec, "Recall:", recall)
        
        if cnt == 0:
            plt.plot(prec, recall, dotshapes[t] + 'k', label=t)
            
        plt.plot(prec, recall, dotshapes[t] + dotColors[cnt])
    
    cnt += 1
    
plt.xlabel("Precision")
plt.ylabel("Recall")
plt.legend()
plt.savefig("precRecVarEval.pdf", format="pdf")
plt.show()
#Make a plot for varying thresholds
fig, ax = plt.subplots()
scrThrs = sorted([0.95, 0.9, 0.8, 0.7])
cmapDsts = plt.get_cmap('winter', 3)
cmapScrs = plt.get_cmap('cool', len(scrThrs))

for t in tpfptnfnVarEval[0.01]:        
    prec = tpfptnfnVarEval[0.01][t]["TPs"] / (tpfptnfnVarEval[0.01][t]["TPs"] + tpfptnfnVarEval[0.01][t]["FPs"])
    recall = tpfptnfnVarEval[0.01][t]["TPs"] / (tpfptnfnVarEval[0.01][t]["TPs"] + tpfptnfnVarEval[0.01][t]["FNs"])
    ax.scatter(prec, recall, marker=dotshapes[t], c='k', label=t)
    
precs = []
recalls = []
shift = (max(scrThrs) - min(scrThrs)) / (len(scrThrs) - 1)
scrThrsColVals = [min(scrThrs) + i * shift for i in range(len(scrThrs))]
padding = (scrThrsColVals[1] - scrThrsColVals[0]) / 2

for th in scrThrs:
    precs.append(tpfptnfnVarScrThr[th]["TPs"] / (tpfptnfnVarScrThr[th]["TPs"] + tpfptnfnVarScrThr[th]["FPs"]))
    recalls.append(tpfptnfnVarScrThr[th]["TPs"] / (tpfptnfnVarScrThr[th]["TPs"] + tpfptnfnVarScrThr[th]["FNs"]))
        
cax = ax.scatter(precs, recalls, marker=dotshapes["ESKEMAP"], c=scrThrs, cmap=cmapScrs, vmin=scrThrsColVals[0] - \
                 padding, vmax=scrThrsColVals[-1] + padding)
cbar = fig.colorbar(cax, ticks=scrThrsColVals, label="ESKEMAP target confidence interval")
cbar.ax.set_yticklabels(scrThrs)
precs = []
recalls = []
dstThrs = sorted([0.03, 0.02, 0.01])
padding = (dstThrs[1] - dstThrs[0]) / 2
        
for th in dstThrs:
    precs.append(tpfptnfnVarDstThr[th]["TPs"] / (tpfptnfnVarDstThr[th]["TPs"] + tpfptnfnVarDstThr[th]["FPs"]))
    recalls.append(tpfptnfnVarDstThr[th]["TPs"] / (tpfptnfnVarDstThr[th]["TPs"] + tpfptnfnVarDstThr[th]["FNs"]))
        
cax = ax.scatter(precs, recalls, marker=dotshapes["edlib"], c=dstThrs, cmap=cmapDsts, vmin=dstThrs[0] - padding,\
                 vmax=dstThrs[2] + padding)
fig.colorbar(cax, ticks=dstThrs, label="Edlib edit distance treshold")
ax.set_xlabel("Precision")
ax.set_ylabel("Recall")
plt.legend()
plt.tight_layout()
plt.savefig("precRecVarEvalAllPnts.pdf", format="pdf")
plt.show()